In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch
from dotenv import load_dotenv
import os


# DB Connection

In [2]:
load_dotenv()

db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

connection = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

conn = connection.cursor()
print("Connection successful!")

Connection successful!


# Read Data

In [3]:
data = pd.read_csv(r'/home/mohamedelawakey/Desktop/Programming Books Recommendation System /ml/data/similarity/v1/books_with_similarity.csv')
data.head()

,Name,Authors,Rating,PublishYear,Publisher,RatingDist5,RatingDist4,RatingDist3,RatingDist2,RatingDist1,...,emb375,emb376,emb377,emb378,emb379,emb380,emb381,emb382,emb383,similarity
0,Between Therapists: The Processing of Transfer...,Arthur Robbins,5.0,1999,Jessica Kingsley Publishers,3,0,0,0,0,...,0.012150,0.032719,0.017120,-0.087718,0.072108,0.046356,0.130460,-0.033658,-0.037323,0.131110
1,Last Word: Media Coverage of the Supreme Court...,Florian Sauvageau,5.0,2005,University of British Columbia Press,1,0,0,0,0,...,0.049502,0.051143,0.057057,-0.051340,0.050311,0.005685,0.018655,0.004255,0.060309,0.063319
2,Autumn Wisdom: A Book of Readings,Richard L. Morgan,5.0,2007,Wipf & Stock Publishers,1,0,0,0,0,...,0.018459,-0.025158,0.031864,-0.049244,-0.088706,0.041197,-0.036828,-0.031374,-0.031394,0.013263
3,James Denney (1856-1917),James M. Gordon,5.0,2006,Wipf & Stock Publishers,1,0,0,0,0,...,0.062438,0.008619,0.072061,-0.063944,0.024584,0.032356,-0.020164,0.008109,-0.042561,0.042210
4,Cautious Rebel: A Biography of Susan Clay Smitzky,Lindsey Apple,5.0,1997,Kent State University Press,1,0,0,0,0,...,-0.054794,0.028854,0.044863,0.041969,-0.036443,0.000149,-0.049355,0.003883,-0.016954,0.024661


In [4]:
for i in data.columns:
    print(i)

Name
Authors
Rating
PublishYear
Publisher
RatingDist5
RatingDist4
RatingDist3
RatingDist2
RatingDist1
RatingDistTotal
CountsOfReview
Description
tech_score
Pages
weighted_rating
average_rating_5
average_rating_4
average_rating_2
average_rating_1
average_high_rating
average_low_rating
Publisher_frequently
Authors_frequently
Pages_scaled
PublishYear_scaled
RatingDistTotal_scaled
tech_score_scaled
CountsOfReview_log
CountsOfReview_scaled
Name_cleaned
Description_cleaned
text_for_embedding
emb0
emb1
emb2
emb3
emb4
emb5
emb6
emb7
emb8
emb9
emb10
emb11
emb12
emb13
emb14
emb15
emb16
emb17
emb18
emb19
emb20
emb21
emb22
emb23
emb24
emb25
emb26
emb27
emb28
emb29
emb30
emb31
emb32
emb33
emb34
emb35
emb36
emb37
emb38
emb39
emb40
emb41
emb42
emb43
emb44
emb45
emb46
emb47
emb48
emb49
emb50
emb51
emb52
emb53
emb54
emb55
emb56
emb57
emb58
emb59
emb60
emb61
emb62
emb63
emb64
emb65
emb66
emb67
emb68
emb69
emb70
emb71
emb72
emb73
emb74
emb75
emb76
emb77
emb78
emb79
emb80
emb81
emb82
emb83
emb84
emb85
emb

# Prepare Data For DB

In [5]:
embedding_cols = [f'emb{i}' for i in range(384)]

data['embedding'] = data[embedding_cols].values.tolist()

/tmp/ipykernel_11140/152140353.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['embedding'] = data[embedding_cols].values.tolist()


In [6]:
data.head()

,Name,Authors,Rating,PublishYear,Publisher,RatingDist5,RatingDist4,RatingDist3,RatingDist2,RatingDist1,...,emb376,emb377,emb378,emb379,emb380,emb381,emb382,emb383,similarity,embedding
0,Between Therapists: The Processing of Transfer...,Arthur Robbins,5.0,1999,Jessica Kingsley Publishers,3,0,0,0,0,...,0.032719,0.017120,-0.087718,0.072108,0.046356,0.130460,-0.033658,-0.037323,0.131110,"[-0.0537614, 0.011557034, -0.024407132, 0.0238..."
1,Last Word: Media Coverage of the Supreme Court...,Florian Sauvageau,5.0,2005,University of British Columbia Press,1,0,0,0,0,...,0.051143,0.057057,-0.051340,0.050311,0.005685,0.018655,0.004255,0.060309,0.063319,"[0.038461395, 0.024522275, -0.042645585, -0.04..."
2,Autumn Wisdom: A Book of Readings,Richard L. Morgan,5.0,2007,Wipf & Stock Publishers,1,0,0,0,0,...,-0.025158,0.031864,-0.049244,-0.088706,0.041197,-0.036828,-0.031374,-0.031394,0.013263,"[-0.018134806, 0.039723296, -0.010537397, 0.08..."
3,James Denney (1856-1917),James M. Gordon,5.0,2006,Wipf & Stock Publishers,1,0,0,0,0,...,0.008619,0.072061,-0.063944,0.024584,0.032356,-0.020164,0.008109,-0.042561,0.042210,"[0.0024561381, -0.049688056, -0.029147396, -0...."
4,Cautious Rebel: A Biography of Susan Clay Smitzky,Lindsey Apple,5.0,1997,Kent State University Press,1,0,0,0,0,...,0.028854,0.044863,0.041969,-0.036443,0.000149,-0.049355,0.003883,-0.016954,0.024661,"[0.008682718, -0.05059993, -0.083236985, -0.06..."


In [7]:
data = data.drop(columns=embedding_cols)

In [8]:
data.head()

,Name,Authors,Rating,PublishYear,Publisher,RatingDist5,RatingDist4,RatingDist3,RatingDist2,RatingDist1,...,PublishYear_scaled,RatingDistTotal_scaled,tech_score_scaled,CountsOfReview_log,CountsOfReview_scaled,Name_cleaned,Description_cleaned,text_for_embedding,similarity,embedding
0,Between Therapists: The Processing of Transfer...,Arthur Robbins,5.0,1999,Jessica Kingsley Publishers,3,0,0,0,0,...,0.090420,0.500168,0.000000,0.0,0.0,Between Therapists: The Processing of Transfer...,Arthur Robbins demonstrates how important coun...,Between Therapists: The Processing of Transfer...,0.131110,"[-0.0537614, 0.011557034, -0.024407132, 0.0238..."
1,Last Word: Media Coverage of the Supreme Court...,Florian Sauvageau,5.0,2005,University of British Columbia Press,1,0,0,0,0,...,0.090722,0.500137,0.049180,0.0,0.0,Last Word: Media Coverage of the Supreme Court...,Media coverage of the Supreme Court of Canada ...,Last Word: Media Coverage of the Supreme Court...,0.063319,"[0.038461395, 0.024522275, -0.042645585, -0.04..."
2,Autumn Wisdom: A Book of Readings,Richard L. Morgan,5.0,2007,Wipf & Stock Publishers,1,0,0,0,0,...,0.090822,0.500137,0.032787,0.0,0.0,Autumn Wisdom: A Book of Readings,"""""Compelling . . . a rare find . . . a very sp...","Autumn Wisdom: A Book of Readings """"Compelling...",0.013263,"[-0.018134806, 0.039723296, -0.010537397, 0.08..."
3,James Denney (1856-1917),James M. Gordon,5.0,2006,Wipf & Stock Publishers,1,0,0,0,0,...,0.090772,0.500137,0.032787,0.0,0.0,James Denney (1856-1917),"James Denney is now best known, though in incr...",James Denney (1856-1917) James Denney is now b...,0.042210,"[0.0024561381, -0.049688056, -0.029147396, -0...."
4,Cautious Rebel: A Biography of Susan Clay Smitzky,Lindsey Apple,5.0,1997,Kent State University Press,1,0,0,0,0,...,0.090320,0.500137,0.016393,0.0,0.0,Cautious Rebel: A Biography of Susan Clay Smitzky,"""Willa Cather wrote that 'the history of every...",Cautious Rebel: A Biography of Susan Clay Smit...,0.024661,"[0.008682718, -0.05059993, -0.083236985, -0.06..."


In [ ]:
data = data[[
    "Name",
    "Name_cleaned",
    "Authors",
    "Publisher",
    "Description",
    "Description_cleaned",
    "Rating",
    "PublishYear",
    "weighted_rating",
    "CountsOfReview_scaled",
    "tech_score_scaled",
    "PublishYear_scaled",
    "average_low_rating",
    "average_high_rating",
    "embedding"
]]

In [ ]:
data.head()

,Name,Name_cleaned,Authors,Publisher,Description,Description_cleaned,Rating,PublishYear,weighted_rating,CountsOfReview_scaled,tech_score_scaled,PublishYear_scaled,average_low_rating,average_high_rating,embedding
0,Between Therapists: The Processing of Transfer...,Between Therapists: The Processing of Transfer...,Arthur Robbins,Jessica Kingsley Publishers,Arthur Robbins demonstrates how important coun...,Arthur Robbins demonstrates how important coun...,5.0,1999,5.0,0.0,0.000000,0.090420,0.0,100.0,"[-0.0537614, 0.011557034, -0.024407132, 0.0238..."
1,Last Word: Media Coverage of the Supreme Court...,Last Word: Media Coverage of the Supreme Court...,Florian Sauvageau,University of British Columbia Press,Media coverage of the Supreme Court of Canada ...,Media coverage of the Supreme Court of Canada ...,5.0,2005,5.0,0.0,0.049180,0.090722,0.0,100.0,"[0.038461395, 0.024522275, -0.042645585, -0.04..."
2,Autumn Wisdom: A Book of Readings,Autumn Wisdom: A Book of Readings,Richard L. Morgan,Wipf & Stock Publishers,"""""Compelling . . . a rare find . . . a very sp...","""""Compelling . . . a rare find . . . a very sp...",5.0,2007,5.0,0.0,0.032787,0.090822,0.0,100.0,"[-0.018134806, 0.039723296, -0.010537397, 0.08..."
3,James Denney (1856-1917),James Denney (1856-1917),James M. Gordon,Wipf & Stock Publishers,"James Denney is now best known, though in incr...","James Denney is now best known, though in incr...",5.0,2006,5.0,0.0,0.032787,0.090772,0.0,100.0,"[0.0024561381, -0.049688056, -0.029147396, -0...."
4,Cautious Rebel: A Biography of Susan Clay Smitzky,Cautious Rebel: A Biography of Susan Clay Smitzky,Lindsey Apple,Kent State University Press,"""Willa Cather wrote that 'the history of every...","""Willa Cather wrote that 'the history of every...",5.0,1997,5.0,0.0,0.016393,0.090320,0.0,100.0,"[0.008682718, -0.05059993, -0.083236985, -0.06..."


In [ ]:
insert_query = """
INSERT INTO books(
    name,
    name_cleaned,
    authors,
    publisher,
    description,
    description_cleaned,
    rating,
    publishyear,
    weighted_rating,
    counts_of_review_scaled,
    tech_score_scaled,
    publishyear_scaled,
    average_low_rating,
    average_high_rating,
    embedding
)
VALUES (
    %(Name)s,
    %(Name_cleaned)s,
    %(Authors)s,
    %(Publisher)s,
    %(Description)s,
    %(Description_cleaned)s,
    %(Rating)s,
    %(PublishYear)s,
    %(weighted_rating)s,
    %(CountsOfReview_scaled)s,
    %(tech_score_scaled)s,
    %(PublishYear_scaled)s,
    %(average_low_rating)s,
    %(average_high_rating)s,
    %(embedding)s
)
"""

In [ ]:
execute_batch(
    conn,
    insert_query,
    data.to_dict(orient='records'),
    page_size=1000

)

connection.commit()